0. Описание задачи

Этот набор данных включает результаты(медали) выступления олимпийских участников и различные личные  факторы(например, пол,рост,вес), которые могут оказывать на результат влияние во время выступления. Так же указаны данные не влияющие на результат. Описание всех признаков будет ниже.

Постановка задачи

Данная задача является задачей классификации олимпийских участников с 4-мя классами(Gold medal, Silver medal, Bronze medal, or NA):

Описание признаков:

Name - имя участника
Sex - пол участника
Age - возраст участника
Height - рост участника
Weight - вес участника
Team - страна
NOC - аббревиатура команды
Games - год и время года
Year - год игр
Season - время года
City - город где проводилось соревнование
Sport - вид спорта
Event - вид спорта с конкретным свойством(например дистанция гонки, круговая она или нет)
Medal - медаль

1. Чтение данных

In [1]:
import numpy as np              # Одномерные и многомерные массивы (array)
import pandas as pd             # Таблицы и временные ряды (dataframe, series)
import matplotlib.pyplot as plt # Научная графика
import seaborn as sns           # Еще больше красивой графики для визуализации данных
import sklearn                  # Алгоритмы машинного обучения
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

url = "https://raw.githubusercontent.com/cherryBepis/ML_Homeworks/master/athlete_events.csv"
x = pd.read_csv(url)


x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [2]:
x.shape

(271116, 15)

In [3]:

x.head(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN
9,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,"Speed Skating Women's 1,000 metres",NaN


2)Переводим признаки типа объекта в категориальные и вычисляем основные числовые характеристики.

In [4]:
x['Name'] = x['Name'].astype('category')
x['Sex'] = x['Sex'].astype('category')
x['Team'] = x['Team'].astype('category')
x['NOC'] = x['NOC'].astype('category')
x['Games'] = x['Games'].astype('category')
x['Season'] = x['Season'].astype('category')
x['City'] = x['City'].astype('category')
x['Sport'] = x['Sport'].astype('category')
x['Event'] = x['Event'].astype('category')
x['Medal'] = x['Medal'].astype('category')
x.drop(x.columns[[0]], axis = 1, inplace = True)
x.describe(include=['float64'])

,Age,Height,Weight
count,261642.000000,210945.000000,208241.000000
mean,25.556898,175.338970,70.702393
std,6.393561,10.518462,14.348020
min,10.000000,127.000000,25.000000
25%,21.000000,168.000000,60.000000
50%,24.000000,175.000000,70.000000
75%,28.000000,183.000000,79.000000
max,97.000000,226.000000,214.000000


Здесь

min – минимальное значение признака в таблице
max – максимальное значение
mean – среднее значение
std – среднеквадратическое отклонение (степень отклонения значений от среднего)
25% – 0.25-квантиль (25-й перцентиль, или первый квартиль) – значение, ниже которого лежит 25% значений
50% – медиана – значение, 50% значений меньше которого, а другие 50% больше
75% – 0.75-квантиль (75-й перцентиль, или третий квартиль) – значение, ниже которого лежит около 75%, т.е. 75% значений меньше данного.

25% участников выступали с возрастом менее 21 года.
25% участников выступали с ростом менее 168 см.
25% участников выступали с весом менее 60 кг.
75% участников выступали с возрастом менее 28 лет.
75% участников выступали с ростом менее 183 см.
75% участников выступали с весом менее 79 кг.

In [5]:
x['Medal'].value_counts()

Medal
Gold      13372
Bronze    13295
Silver    13116
Name: count, dtype: int64

Не получили медали - 231333 участника(учитываться могут одни и те же участники в одном спорте с разным свойством(длина,тип гонки и т.д)).

Построим матрицу корелляции для числовых признаков:

In [6]:
corrMatrix = x.select_dtypes(include=['float64']).corr()
corrMatrix.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)

,Age,Height,Weight
Age,1.000000,0.138246,0.212069
Height,0.138246,1.000000,0.796213
Weight,0.212069,0.796213,1.000000


Из данной матрицы можем заметить, что

Есть небольшая корреляция между весом и ростом, но на основе этого нельзя сделать никаких толковых выводов.

Информация о категориальных признаках

In [7]:
x.describe(include=['category'])

,Name,Sex,Team,NOC,Games,Season,City,Sport,Event,Medal
count,271116,271116,271116,271116,271116,271116,271116,271116,271116,39783
unique,134732,2,1184,230,51,2,42,66,765,3
top,Robert Tait McKenzie,M,United States,USA,2000 Summer,Summer,London,Athletics,Football Men's Football,Gold
freq,58,196594,17847,18853,13821,222552,22426,38624,5733,13372


Большинство участников - мужчины.
Больше всего было получено золотых медалей.
Больше всего игр было проведено в США(так оно и есть на самом деле)

6)Разбиение данных на обучающую и тестовую выборки

In [8]:
y = x['Medal']
Xclear = x.drop(columns=['Medal'])
X_train, X_test, y_train, y_test = train_test_split(Xclear, y, test_size = 0.25, random_state = 11) # 42 было

N_train, _ = X_train.shape
N_test,  _ = X_test.shape

print("Разбили следующим образом:")
N_train, N_test

Разбили следующим образом:


(203337, 67779)

3) Пропущенные значения

In [9]:
x.isnull().sum()

Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

Строки с пропущенными значениями просто удалим. Не то что бы это надо делать конкретно с данным дата сетом, тк вес росит и возраст не особо важные признаки. Но удалим все те, где не указан возраст

In [10]:
[feature for feature in x.columns if (x[feature].isnull().sum()>1 and x[feature].isnull().sum()<60000) ]

['Age']

In [11]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   Name    271116 non-null  category
 1   Sex     271116 non-null  category
 2   Age     261642 non-null  float64 
 3   Height  210945 non-null  float64 
 4   Weight  208241 non-null  float64 
 5   Team    271116 non-null  category
 6   NOC     271116 non-null  category
 7   Games   271116 non-null  category
 8   Year    271116 non-null  int64   
 9   Season  271116 non-null  category
 10  City    271116 non-null  category
 11  Sport   271116 non-null  category
 12  Event   271116 non-null  category
 13  Medal   39783 non-null   category
dtypes: category(10), float64(3), int64(1)
memory usage: 17.5 MB


In [12]:
x=x.dropna()

In [13]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30181 entries, 40 to 271103
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Name    30181 non-null  category
 1   Sex     30181 non-null  category
 2   Age     30181 non-null  float64 
 3   Height  30181 non-null  float64 
 4   Weight  30181 non-null  float64 
 5   Team    30181 non-null  category
 6   NOC     30181 non-null  category
 7   Games   30181 non-null  category
 8   Year    30181 non-null  int64   
 9   Season  30181 non-null  category
 10  City    30181 non-null  category
 11  Sport   30181 non-null  category
 12  Event   30181 non-null  category
 13  Medal   30181 non-null  category
dtypes: category(10), float64(3), int64(1)
memory usage: 6.7 MB
